In [1]:
import numpy as np
import pandas as pd

keys = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label.']

kdd_df = pd.read_csv('/Users/sawyer/Downloads/assignment_3/kddcup.csv', names=keys)
test = pd.read_csv('/Users/sawyer/Downloads/assignment_3/corrected.csv', names=keys)
kdd_df


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label.
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [31]:
import sklearn.ensemble
import sklearn.preprocessing

def preprocess_df(df):
    keys = list(df)
    X = df[keys[:-1]]
    y = df[keys[-1]]
    # nominal string features
    one_hot_features = [1, 2, 3]
    one_hot_columns = [keys[i] for i in one_hot_features]
    
    preprocessed_X = pd.get_dummies(X, dummy_na=False, columns=one_hot_columns)
    # multi class classifier (1 for malicious)
    l_encoder = sklearn.preprocessing.LabelEncoder()
    out_y = l_encoder.fit_transform(y)
    return preprocessed_X, out_y, l_encoder
X, y, tr_l_encoder = preprocess_df(kdd_df)
y

array([11, 11, 11, ..., 11, 11, 11])

In [32]:
X_test, y_test, te_l_encoder = preprocess_df(test)

for key in set(X):
    if key not in X:
        X[key] = 0

X_test = X_test.reindex(columns=X.columns, fill_value=0)

y_uniq = {}
for v in y:
    k = tr_l_encoder.classes_[v]
    if k not in y_uniq:
        y_uniq[k] = len(y_uniq)
print(len(y_uniq))
for v in y_test:
    k = te_l_encoder.classes_[v]
    if k not in y_uniq:
        y_uniq[k] = len(y_uniq)
print(y_uniq)
# use y's label encodings
y = [y_uniq[tr_l_encoder.classes_[v]] for v in y]
y_test = [y_uniq[te_l_encoder.classes_[v]] for v in y_test]
        

most_common = max(set(y_test), key=y.count)
print(max(set(y_test), key=y_test.count))
print(most_common)

23
{'normal.': 0, 'buffer_overflow.': 1, 'loadmodule.': 2, 'perl.': 3, 'neptune.': 4, 'smurf.': 5, 'guess_passwd.': 6, 'pod.': 7, 'teardrop.': 8, 'portsweep.': 9, 'ipsweep.': 10, 'land.': 11, 'ftp_write.': 12, 'back.': 13, 'imap.': 14, 'satan.': 15, 'phf.': 16, 'nmap.': 17, 'multihop.': 18, 'warezmaster.': 19, 'warezclient.': 20, 'spy.': 21, 'rootkit.': 22, 'snmpgetattack.': 23, 'named.': 24, 'xlock.': 25, 'xsnoop.': 26, 'sendmail.': 27, 'saint.': 28, 'apache2.': 29, 'udpstorm.': 30, 'xterm.': 31, 'mscan.': 32, 'processtable.': 33, 'ps.': 34, 'httptunnel.': 35, 'worm.': 36, 'mailbomb.': 37, 'sqlattack.': 38, 'snmpguess.': 39}
5
5


In [33]:
model = sklearn.ensemble.RandomForestClassifier()
model.fit(X, y)

RandomForestClassifier()

In [36]:
#y_softmax = model.predict_proba(X_test)
y_hat = model.predict(X_test)



#threshold = 0.00000001
#y_hat = [1 if y >= threshold else 0 for _, y in y_softmax]
#print(y_softmax)
#print(sum([1 for _, y in y_softmax if y >= threshold]))
#print(len(y_softmax))
y_test - y_hat
#y_hat

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
n_correct = sum([1 if y_test[i] == y_hat[i] else 0 for i in range(len(y_test))])
print(n_correct/len(y_test))

    

0.919280838764231
